# Midterm take-home

<p>Instructions
<ol>
<li>Download the files <i>CCARs.xlsx</i> and <i>Services.xlsx</i> into the same folder as this notebook.
<li>Read the file <i>Documentation.pdf</i>
<li>Run the code below and then answer the questions
</ol></p>

<b>Penalties:</b> You will incur penalties if:
<ul>
<li>Your code is wrong</li>
<li>Your code does not work on different data</li>
<li>You use for loops (unless explicitly permitted)</li>
</ul>

In [21]:
import xlrd # you might need to install this package (either via Canopy or by executing "pip install xlrd" in a terminal)
import pandas as pd
ccars = pd.read_excel('CCARs.xlsx')
services = pd.read_excel('Services.xlsx')
# the code above will take a long time to run, because the files are large

Here are some CCAR.

In [4]:
ccars.head()

,CLIENT_ID,CCAR_TYPE,CCAR_DATE,HOUSING,LEVEL_OF_FUNCTIONING,SYMPTOM_SEVERITY,HOPE
0,4A8998A4966913,NaN,2015-04-15,15,3,3,1
1,A628826A899812,NaN,2015-04-15,12,6,6,6
2,44277244933913,NaN,2015-04-15,14,5,5,2
3,A153351A411412,NaN,2015-04-15,15,3,6,2
4,22655622722713,1.0,2015-04-15,15,3,3,3


Here are some services.

In [5]:
services.head(5)

,CLIENT_ID,MBR_AGE,MBR_GENDER,MBR_PRIMARY_LANGUAGE,MBR_ZIPCODE,MEDICAID_ELIG_CAT,SERVICE_DATE,SERVICE_CODE,SERV_CODE_DESCRP,DIAGNOSIS,...,SERVICE_TYPE,SERVICE_DURATION,ADMISSION_DATE,DISCHARGE_DATE,PROVIDER,PRV_TYP_DESC,PROV_CREDENTIALS,PROV_ZIPCODE,PROV_FLAG,SOURCE
0,69266296A66A31,40,Male,ENGLISH,80304,DISABLED,2015-01-01,H0018,Behavioral health; short-term residential (non...,Other Schizoaffective Disorders,...,ATU,1260,,,MENTAL HEALTH PARTNERS,COMMUNITY MENTAL HEALTH CENTER OR PUB HLTH AGENCY,CAC III,80304,MHP CMHC,Encounter
1,58777785A44A11,28,Male,ENGLISH,80302,DISABLED,2015-01-01,H0018,Behavioral health; short-term residential (non...,Other Schizoaffective Disorders,...,ATU,360,,,MENTAL HEALTH PARTNERS,COMMUNITY MENTAL HEALTH CENTER OR PUB HLTH AGENCY,CAC III,80304,MHP CMHC,Encounter
2,87855878244282,20,Female,ENGLISH,80504,DISABLED,2015-01-01,H0018,Behavioral health; short-term residential (non...,Other Schizoaffective Disorders,...,ATU,1310,,,MENTAL HEALTH PARTNERS,COMMUNITY MENTAL HEALTH CENTER OR PUB HLTH AGENCY,CAC III,80304,MHP CMHC,Encounter
3,49A88A94244201,36,Male,ENGLISH,80304,DISABLED,2015-01-01,H0018,Behavioral health; short-term residential (non...,"Schizophrenia, Unspecified",...,ATU,1440,,,MENTAL HEALTH PARTNERS,COMMUNITY MENTAL HEALTH CENTER OR PUB HLTH AGENCY,CAC III,80304,MHP CMHC,Encounter
4,7A8778A7799771,45,Female,ENGLISH,80302,DISABLED,2015-01-01,H0018,Behavioral health; short-term residential (non...,"Schizophrenia, Unspecified",...,ATU,1080,,,MENTAL HEALTH PARTNERS,COMMUNITY MENTAL HEALTH CENTER OR PUB HLTH AGENCY,CAC III,80304,MHP CMHC,Encounter


In [6]:
len(ccars)

15079

In [7]:
len(services)

335880

## Warm-up (not graded)

<p>
These two questions are not graded. They are just to make sure that you understand the data sets.
</p>

#### What's the proportion of services that are claims (see column <i>Source</i> of services)? 

In [8]:
(services['SOURCE'] == 'Claim').mean()

0.33151423127307372

The correct answer is 0.33151423127307372

#### For every Medicaid category (MEDICAID_ELIG_CAT), find the average level of Hope recorded for clients in that Medicaid category. 

In [9]:
ccars.merge(services[['CLIENT_ID','MEDICAID_ELIG_CAT']],left_on='CLIENT_ID',right_on='CLIENT_ID')\
.groupby('MEDICAID_ELIG_CAT')['HOPE'].agg('mean')

MEDICAID_ELIG_CAT
ADULT          3.194272
AwDC           3.390436
CHILD          3.005202
DISABLED       3.531640
ELDERLY        3.358147
FOSTER CARE    3.205073
OTHER          2.845111
Name: HOPE, dtype: float64

The correct answer is as follows:

MEDICAID_ELIG_CAT<br/>
ADULT &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;        3.194272<br/>
AwDC    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        3.390436<br/>
CHILD    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;      3.005202<br/>
DISABLED  &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    3.531640<br/>
ELDERLY    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;    3.358147<br/>
FOSTER CARE &nbsp;&nbsp;&nbsp;   3.205073<br/>
OTHER    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;       2.845111<br/>
Name: HOPE, dtype: float64

### Questions (10 pts)

As explained in the <i>Documentation</i>, services (i.e., rows in the services DataFrame) can be either encounters or claims. It is desirable to limit the number of claims, which are more expensive than encounters. Our goal is to manually explore the data in order to find possible drivers of claims.

#### Question 1 (1 pt):

For each type of service (Encounter vs Claim), report the number of services in that group and their average charged amount. Report your results in a single table.

In [23]:
services.groupby('SOURCE')['CHARGED_AMT'].agg({'No. of services':'size','Avg. charged amount' : 'mean'})

,Avg. charged amount,No. of services
SOURCE,,
Claim,326.208942,111349
Encounter,184.867381,224531


#### Question 2 (1 pt):

For each DIAGNOSIS, compute the number of services with that diagnosis and the proportion (0 to 1, computed among the services with that diagnosis) of services that are claims. Visualize only diagnoses with at least 10,000 services sorted by decreasing proportion of claims. Report your results in a single table.

In [24]:
diag = services[['DIAGNOSIS','SOURCE']]
diag['SERVICES'] = (diag['SOURCE'] == 'Claim')
gb = diag.groupby('DIAGNOSIS')['SERVICES']
diag = gb.agg({'Count' : 'count','Proportion' : 'mean'})
diag.loc[diag['Count'] >= 10000].sort_values(['Proportion'],ascending=False)

C:\Users\ronathan\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Count,Proportion
DIAGNOSIS,,
"Opioid Dependence, Uncomplicated",63832,0.953127
"Alcohol Dependence, Uncomplicated",18466,0.195549
"Post-Traumatic Stress Disorder, Unspecified",19911,0.125358
Generalized Anxiety Disorder,11330,0.118094
"Post-Traumatic Stress Disorder, Acute",10413,0.077595
"Major Depressive Affective Disorder, Recurrent Episode, Moderate",14347,0.076044
"Schizophrenia, Unspecified",12444,0.030135
Other Schizoaffective Disorders,11172,0.017633
"Schizoaffective Disorder, Bipolar Type",13782,0.015745


#### Question 3 (3 pts):

We want to see if a client's CCAR reveals whether the client will soon have a Claim. To this end, build a DataFrame, called <b>df</b> with one row per CCAR* and the following columns (with <b>exactly</b> these names):
<ul> 
<li><b>CLIENT_ID</b>: the client id
<li><b>CCAR_DATE</b>: the CCAR date
<li><b>SYMPTOM_SEVERITY</b>: the SYMPTOM_SEVERITY recorded in the CCAR
<li><b>DAYS_TO_SERVICE</b>: the days elapsed between the CCAR and the client's next service. By "next service", we mean the first service involving the same client after the client's CCAR. 
<li><b>CLAIM</b>: True (or 1) if the type of the next service is a Claim, False (or 0) if it is an Encounter. 
</ul>


\* Exclude from your analysis those CCARs that are not followed by any service for that client. For simplicity, assume that a client can have at most one CCAR in a day. Thus, your DataFrame df should have one row for each combination of CLIENT_ID and CCAR_DATE.

In [25]:
"""
Merging ccars and services on client_id - inner join since we are not interested in clients who have not gone for any service 
(not present in services) or clients who have gone for service but have not been assessed (not present in CCARS)
"""
df = ccars[['CLIENT_ID','CCAR_DATE','SYMPTOM_SEVERITY']]\
.merge(services[['CLIENT_ID','SERVICE_DATE','SOURCE']],left_on='CLIENT_ID',right_on='CLIENT_ID')

df['DAYS_TO_SERVICE'] = df['SERVICE_DATE'] - df['CCAR_DATE']

df = df[df['DAYS_TO_SERVICE'] > '0'].reset_index(drop=True)

df['CLAIM'] = (df['SOURCE'] == 'Claim')
# Dropping unwanted columns in the dataframe
df.drop(['SERVICE_DATE','SOURCE'],axis=1,inplace=True)


# Retaining only rows where the days to service is minimum for a group of (CLIENT_ID,CCAR_DATE)

df=df.groupby(['CLIENT_ID','CCAR_DATE'],as_index=False).apply(lambda d : d[d.DAYS_TO_SERVICE == d.DAYS_TO_SERVICE.min()])

df=df.drop_duplicates(['CLIENT_ID','CCAR_DATE','DAYS_TO_SERVICE'], keep='first')

In [26]:
df.head()

,,CLIENT_ID,CCAR_DATE,SYMPTOM_SEVERITY,DAYS_TO_SERVICE,CLAIM
0,276362,11111111699652,2016-06-17,5,20 days,False
1,161866,11155111699612,2015-09-24,2,111 days,False
2,210115,11166111388352,2015-06-16,5,14 days,False
3,210142,11166111388352,2016-06-20,5,24 days,False
4,173968,11199111533531,2015-01-23,5,13 days,False


In [27]:
len(df)

9881

#### Question 3.1 (1 pt)

Show only the rows of <i>df</i> relative to client 118118119AA962

In [28]:
df[df['CLIENT_ID'] == '118118119AA962']

,,CLIENT_ID,CCAR_DATE,SYMPTOM_SEVERITY,DAYS_TO_SERVICE,CLAIM
102,285692,118118119AA962,2015-10-23,5,160 days,False
103,285691,118118119AA962,2016-10-07,5,46 days,True
104,285690,118118119AA962,2016-10-14,5,39 days,True


Use df to answer <b>all following questions</b>. 

#### Question 4 (1 pt):

What's the probability that a CCAR is followed by a claim within 30 days?

In [29]:
((df['DAYS_TO_SERVICE'] <= '30 days') & (df['CLAIM'] == True)).mean()

0.078736969942313537

#### Question 5 (2 pts):

What's the probability that a CCAR with SYMPTOM_SEVERITY $>$ 7 is followed by a claim within 30 days?  What's the probability that a CCAR assessment with SYMPTOM_SEVERITY $<$ 3 is followed by a claim within 30 days? 

In [30]:
symptom7_30 = ((df['DAYS_TO_SERVICE'] <= '30 days') & (df['CLAIM'] == True) & (df['SYMPTOM_SEVERITY'] > 7)).mean()
symptom3_30 = ((df['DAYS_TO_SERVICE'] <= '30 days') & (df['CLAIM'] == True) & (df['SYMPTOM_SEVERITY'] < 3)).mean()
print "The probability that a CCAR with SYMPTOM_SEVERITY > 7 is followed by a claim within 30 days is %f"%symptom7_30
print "The probability that a CCAR with SYMPTOM_SEVERITY < 3 is followed by a claim within 30 days is %f"%symptom3_30

The probability that a CCAR with SYMPTOM_SEVERITY > 7 is followed by a claim within 30 days is 0.002530
The probability that a CCAR with SYMPTOM_SEVERITY < 3 is followed by a claim within 30 days is 0.003542


#### Question 6 (1 pt):

Compute the 95% confidence intervals of the proportions you found in Question 5 to assess whether those two proportions are significantly different from each other. The last section of the notebook shows how to compute the confidence interval of a proportion. Briefly discuss what your results suggest.

In [31]:
import statsmodels.stats.proportion
prop1 = ((df['DAYS_TO_SERVICE'] <= '30 days') & (df['CLAIM'] == True) & (df['SYMPTOM_SEVERITY'] > 7)).sum()
prop2 = ((df['DAYS_TO_SERVICE'] <= '30 days') & (df['CLAIM'] == True) & (df['SYMPTOM_SEVERITY'] < 3)).sum()
total = len(df)
conf1 = statsmodels.stats.proportion.proportion_confint(count=prop1,nobs=total)
conf2 = statsmodels.stats.proportion.proportion_confint(count=prop2,nobs=total)
print "The 95%% confidence intervals for the first proportion is (%f,%f)"%conf1
print "The 95%% confidence intervals for the second proportion is (%f,%f)"%conf2

The 95% confidence intervals for the first proportion is (0.001540,0.003521)
The 95% confidence intervals for the second proportion is (0.002371,0.004714)


In [32]:
print "From the confidence intervals above, it appears that the two probabilities are not significantly different\
 from each other since probability 1 lies within the confidence interval of probability 2."
print "\nThis suggests that the probability of a CCAR followed by a claim within 30 days does not significantly vary by\
 how low or high the severity of the symptom is. A high severity symptom has almost similar probability of a claim within\
 30 days as a low severity symptom."

From the confidence intervals above, it appears that the two probabilities are not significantly different from each other since probability 1 lies within the confidence interval of probability 2.

This suggests that the probability of a CCAR followed by a claim within 30 days does not significantly vary by how low or high the severity of the symptom is. A high severity symptom has almost similar probability of a claim within 30 days as a low severity symptom.


#### Finding the confidence interval of a proportion

To find a confidence interval of a proportion, the first thing to do is to install the package <i>statsmodel</i>. Then, consider the following example.

Example: during the exit polls for a certain election, we interviewed s sample of 40 voters and found that 23 of them voted for candidate A (a proportion of 0.575). What's the confidence interval of this proportion? Is the proportion significantly greater than 0.50?

In [20]:
#import statsmodels.stats.proportion
statsmodels.stats.proportion.proportion_confint(count=23,nobs=40) # by default, it's a 95% confidence interval

(0.42180433268960182, 0.72819566731039809)

The confidence interval is [0.42 - 0.73]. The observed proportion is not significantly higher than 50%.